In [16]:
import numpy as np
from sklearn.model_selection import train_test_split  #数据集的分割函数,就是把一组数据分成训练集合测试集
from sklearn.preprocessing import StandardScaler      #数据预处理

#引入包含数据验证方法的包，就是为了来计算准确率的
from sklearn import metrics

class SingeHiddenLayer(object):  #定义单隐藏层这个类
    
    def __init__(self,X,y,num_hidden):#进行初始化，然后进行实例化
        self.data_x = np.atleast_2d(X)       #判断输入训练集是否大于等于二维; 把x_train()取下来
        #numpy.atleast_xd()  atleast_xd 支持将输入数据直接视为 x维
        self.data_y = np.array(y).flatten()  #a.flatten()把a放在一维数组中，不写参数默认是“C”，也就是先行后列的方式，也有“F”先列后行的方式； 把 y_train取下来
        self.num_data = len(self.data_x)  #训练数据个数
        self.num_feature = self.data_x.shape[1];  #shape[] 读取矩阵的长度，比如shape[0]就是读取矩阵第一维度的长度 (120行，4列，所以shape[0]==120,shapep[1]==4)
        self.num_hidden = num_hidden;  #隐藏层节点个数
        
        #随机生产权重（从-1，到1，生成（num_feature行,num_hidden列））
        self.w = np.random.uniform(-1, 1, (self.num_feature, self.num_hidden)) 
        print(np.shape(self.w))
        
        #随机生成偏置，一个隐藏层节点对应一个偏置
        #for i in range(self.num_hidden):
        b = np.random.uniform(-0.6, 0.6, (1, self.num_hidden))
        self.first_b = b
   
        #生成偏置矩阵，以隐藏层节点个数4为行，样本数120为列
        for i in range(self.num_data-1):
            b = np.row_stack((b, self.first_b))  #row_stack 以叠加行的方式填充数组
        self.b = b
        print("偏置是:",np.shape(self.b))
    #定义sigmoid函数  
    def sigmoid(self,x):
        return 1.0 / (1 + np.exp(-x))
    
    
     
            
    def train(self,x_train,y_train,classes):#定义训练方法
        mul = np.dot(self.data_x, self.w)    #输入乘以权重 ,也就是H*beta
        add = mul + self.b                        #加偏置 add表示输出神经元
        H = self.sigmoid(add)                #激活函数  使用激活函数

        H_ = np.linalg.pinv(H)               #求H的广义逆矩阵H_相当论文的于H+
        print(type(H_.shape))                 #避免使用np
        
        #将只有一列的Label矩阵转换，例如，iris的label中共有三个值，则转换为3列，以行为单位，label值对应位置标记为1，其它位置标记为0

        self.train_y = np.zeros((self.num_data,classes))  #初始化一个120行，3列的全0矩阵
        print("标签的形状",np.shape(self.train_y) )
        for i in range(0,self.num_data):
           # print("ruru",y_train[i])
            self.train_y[i,y_train[i]] = 1   #对应位置标记为1  ,y_train干嘛嘞？没整太明白？？？？ onehot编码
            
            
        
        self.out_w = np.dot(H_,self.train_y)  #求输出权重  相当于第二个beta
        
    def predict(self,x_test):
        self.t_data = np.atleast_2d(x_test)    #测试数据集
        self.num_tdata = len(self.t_data)      #测试集的样本数
        self.pred_Y = np.zeros((x_test.shape[0]))  #对测试集进行初始化
        
        b = self.first_b
        
        #扩充偏置矩阵，以隐藏层节点个数4为行，样本数30为列
        for i in range(self.num_tdata-1):
            b = np.row_stack((b, self.first_b))  #以叠加行的方式填充数组
          
        #预测  ，最小beta的输出
        self.pred_Y = np.dot(self.sigmoid(np.dot(self.t_data,self.w)+b),self.out_w) #普通用乘，矩阵用点积
        
        #取输出节点中值最大的类别作为预测值 
        self.predy = []
        for i in self.pred_Y:
            L = i.tolist() #将i这个数组转换成列表
           # print(L)
            self.predy.append(L.index(max(L)))   #索引到输出节点中最大的值

    def score(self,y_test):
        print("准确率：")
        #使用准确率方法验证
        print(metrics.accuracy_score(y_true=y_test,y_pred=self.predy)) 
        #调用metrics.accuracy_score方法来计算分类的准确率
        
stdsc = StandardScaler()   #StandardScaler类,利用接口在训练集上计算均值和标准差，以便于在后续的测试集上进行相同的缩放
X = np.loadtxt("mnist2500_X.txt")
labels = np.loadtxt("mnist2500_labels.txt")  #进行实例化
labels = list(map(int, labels))  #完美呀，因为鸢尾花给的是整型，而mnist2500是浮点型(float)而导致这种错误

#X = np.vectorize(X) #np.int()作用的对象是一个值而不是一个数组
#labels = np.vectorize(labels)
x,y = stdsc.fit_transform(X),labels   #数据归一化，y表示的就是标签
#x = np.int(x)
#y = np.int(y)
x_train, x_test, y_train, y_test = train_test_split(x , y, test_size=0.2, random_state=0)


ELM = SingeHiddenLayer(x_train,y_train,100)   #训练数据集，训练集的label，隐藏层节点个数
ELM.train(x_train,y_train,10) #y_train表示训练集的label
ELM.predict(x_test)#x_test这个数据集就是用来预测的
ELM.score(y_test)


(784, 100)
偏置是: (2000, 100)
<class 'tuple'>
标签的形状 (2000, 10)
准确率：
0.764


In [13]:
type(y_train)

list

In [3]:
np.set_printoptions(threshold=np.inf)
print(labels)

[5. 0. 4. 1. 9. 2. 1. 3. 1. 4. 3. 5. 3. 6. 1. 7. 2. 8. 6. 9. 4. 0. 9. 1.
 1. 2. 4. 3. 2. 7. 3. 8. 6. 9. 0. 5. 6. 0. 7. 6. 1. 8. 7. 9. 3. 9. 8. 5.
 9. 3. 3. 0. 7. 4. 9. 8. 0. 9. 4. 1. 4. 4. 6. 0. 4. 5. 6. 1. 0. 0. 1. 7.
 1. 6. 3. 0. 2. 1. 1. 7. 9. 0. 2. 6. 7. 8. 3. 9. 0. 4. 6. 7. 4. 6. 8. 0.
 7. 8. 3. 1. 5. 7. 1. 7. 1. 1. 6. 3. 0. 2. 9. 3. 1. 1. 0. 4. 9. 2. 0. 0.
 2. 0. 2. 7. 1. 8. 6. 4. 1. 6. 3. 4. 5. 9. 1. 3. 3. 8. 5. 4. 7. 7. 4. 2.
 8. 5. 8. 6. 7. 3. 4. 6. 1. 9. 9. 6. 0. 3. 7. 2. 8. 2. 9. 4. 4. 6. 4. 9.
 7. 0. 9. 2. 9. 5. 1. 5. 9. 1. 2. 3. 2. 3. 5. 9. 1. 7. 6. 2. 8. 2. 2. 5.
 0. 7. 4. 9. 7. 8. 3. 2. 1. 1. 8. 3. 6. 1. 0. 3. 1. 0. 0. 1. 7. 2. 7. 3.
 0. 4. 6. 5. 2. 6. 4. 7. 1. 8. 9. 9. 3. 0. 7. 1. 0. 2. 0. 3. 5. 4. 6. 5.
 8. 6. 3. 7. 5. 8. 0. 9. 1. 0. 3. 1. 2. 2. 3. 3. 6. 4. 7. 5. 0. 6. 2. 7.
 9. 8. 5. 9. 2. 1. 1. 4. 4. 5. 6. 4. 1. 2. 5. 3. 9. 3. 9. 0. 5. 9. 6. 5.
 7. 4. 1. 3. 4. 0. 4. 8. 0. 4. 3. 6. 8. 7. 6. 0. 9. 7. 5. 7. 2. 1. 1. 6.
 8. 9. 4. 1. 5. 2. 2. 9. 0. 3. 9. 6. 7. 2. 0. 3. 5.

In [4]:
print(len(labels))

2500


In [9]:
labels = list(map(int, labels))
labels

[5,
 0,
 4,
 1,
 9,
 2,
 1,
 3,
 1,
 4,
 3,
 5,
 3,
 6,
 1,
 7,
 2,
 8,
 6,
 9,
 4,
 0,
 9,
 1,
 1,
 2,
 4,
 3,
 2,
 7,
 3,
 8,
 6,
 9,
 0,
 5,
 6,
 0,
 7,
 6,
 1,
 8,
 7,
 9,
 3,
 9,
 8,
 5,
 9,
 3,
 3,
 0,
 7,
 4,
 9,
 8,
 0,
 9,
 4,
 1,
 4,
 4,
 6,
 0,
 4,
 5,
 6,
 1,
 0,
 0,
 1,
 7,
 1,
 6,
 3,
 0,
 2,
 1,
 1,
 7,
 9,
 0,
 2,
 6,
 7,
 8,
 3,
 9,
 0,
 4,
 6,
 7,
 4,
 6,
 8,
 0,
 7,
 8,
 3,
 1,
 5,
 7,
 1,
 7,
 1,
 1,
 6,
 3,
 0,
 2,
 9,
 3,
 1,
 1,
 0,
 4,
 9,
 2,
 0,
 0,
 2,
 0,
 2,
 7,
 1,
 8,
 6,
 4,
 1,
 6,
 3,
 4,
 5,
 9,
 1,
 3,
 3,
 8,
 5,
 4,
 7,
 7,
 4,
 2,
 8,
 5,
 8,
 6,
 7,
 3,
 4,
 6,
 1,
 9,
 9,
 6,
 0,
 3,
 7,
 2,
 8,
 2,
 9,
 4,
 4,
 6,
 4,
 9,
 7,
 0,
 9,
 2,
 9,
 5,
 1,
 5,
 9,
 1,
 2,
 3,
 2,
 3,
 5,
 9,
 1,
 7,
 6,
 2,
 8,
 2,
 2,
 5,
 0,
 7,
 4,
 9,
 7,
 8,
 3,
 2,
 1,
 1,
 8,
 3,
 6,
 1,
 0,
 3,
 1,
 0,
 0,
 1,
 7,
 2,
 7,
 3,
 0,
 4,
 6,
 5,
 2,
 6,
 4,
 7,
 1,
 8,
 9,
 9,
 3,
 0,
 7,
 1,
 0,
 2,
 0,
 3,
 5,
 4,
 6,
 5,
 8,
 6,
 3,
 7,
 5,
 8,
 0,
 9,
 1,
 0,


In [17]:
print(np.shape(x_train))

(2000, 784)


In [18]:
print(len(X))
print(np.shape(X))

2500
(2500, 784)


In [19]:
print(X)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print(y_test)
print(np.shape(y_test))

In [ ]:
print(x)

In [ ]:
print(type(x))
print(type(y))

In [ ]:
print(np.shape(X))
print(np.shape(labels))
print(labels)

In [ ]:
print(labels)